### Goal: 
<br> 1. obtain  information about stocks from S&P 500.
<br> 2. retrive fundanemtal  data of S&P 500 stocks  from Qandle.
<br> 3. Aquire pricing info for list of stocks from yFinance
<br> 4. Combining data together

In [1]:

import requests
import pandas as pd
import numpy as np
from datetime import date, datetime
from pandas.tseries.offsets import BDay
import sys, os
import pickle

import quandl
import yfinance as yf

from pandas_datareader import data as pdr



In [39]:
from urllib.request import urlopen
import warnings
import json
warnings.filterwarnings('ignore')
# enable pandas to show all the column
pd.set_option('display.max_columns', None)

In [2]:
# API Credentials
quandl.ApiConfig.api_key = '*****************'

In [3]:
parentDirectory = os.getcwd()

In [4]:
# S&P 500 Scraper
def sp500_list_retrieval():
    """
    Obtaine:
    - S&P 500 stock info from Wikipedia page.
    Returns:
    - a dataframe of all tickers and corresponding industry information.
    """
    print("Acquiring S&P500 Tickers info...")

    table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

    df = table[0]
    df.columns = ['Symbol',
                  'Security',
                  'SEC filings',
                  'GICS Sector',
                  'GICS Sub Industry',
                  'Headquarters Location',
                  'Date First Added',
                  'CIK',
                  'Founded'
                   ]
    df.drop(labels=['SEC filings',
                    'Headquarters Location',
                    'Date First Added',
                    'CIK'
                    ],
              axis=1,
              inplace=True
              )

    df.to_csv(f"{parentDirectory}/data/SP500_list.csv")
    return df



In [5]:
# Fundamental Data Pull from Quandl

def fundamentals_data_pull(timeframe='MRY', ticker=[], start_yr='2014', end_date=None):
    """Fetch fundamentals data for each ticker. By default  all timeframes (dimensions) will be fetched. 
    Otherwise, timeframes may be selected from one of the following:
    Select One:
    - AR (As Reported)
    - MR (Most-Recent Reported)
    + One of the Following
    - Y (Annual)
    - T (Trailing Twelve Months)
    - Q (Quarterly)
    (i.e. MRY = Most-Recent Reported Annual Data)
    
    Keyword Arguments:
    timeframe -- one of the above (default 'MRY')
    ticker -- list of stock tickers (default empty list)
    Returns:
    dict -- keys are tickers, values are dataframe of all Quandl fundamentals data 
    """

    print("Obtaining fundamentals data. ")
    
    total_length = 0

    # Reduce dataset only to years requested
    cutoff_date = str(start_yr + "-01-01")
    
    if timeframe == None:
        timeframe = input("Please select a timeframe from 'MRT', 'ART','ARY', or 'MRY':  ")
        #df = quandl.get_table('SHARADAR/SF1', ticker=ticker)
        # Removing to eliminate confusion with Quarterly results and potential incompatibility down the line -- AB 9/14

    elif timeframe in ['MRT', 'ART','ARY', 'MRY']:
        if end_date == None:
            end_date = str(date.today())
            df = quandl.get_table('SHARADAR/SF1', dimension=timeframe, calendardate={'gte': cutoff_date, 'lte' : end_date}, ticker=ticker, paginate=True)
        else:
            df = quandl.get_table('SHARADAR/SF1', dimension=timeframe, calendardate={'gte': cutoff_date, 'lte' : end_date}, ticker=ticker, paginate=True)

    elif timeframe in ['MRQ', 'ARQ']:
        raise ValueError("Quarterly data is not compatible with this analysis. Please select a timeframe from 'MRT', 'ART','ARY', or 'MRY'")

    else:
        timeframe = input("Please select a timeframe from 'MRT', 'ART','ARY', or 'MRY':  ")


    # Create a dictionary where keys are tickers and values are dataframes of fundamentals
    fund_dict = {}
    for x in ticker:
        df0 = df.copy()
        df0 = df0[df0['ticker'] == x.upper()]
        if len(df0) == 0:
            print("No data provided for symbol '" + x.upper() + "'")
            pass
        else:
            fund_dict[x.upper()] = df0
            total_length += len(df0)

    print("total length of fundamentals data dict: " + str(len(fund_dict.keys())))
    print("total length of fundamentals data: " + str(total_length))
    return  fund_dict 



In [6]:
#quandl.get_table('SHARADAR/SF1')

In [7]:
# stock_list = list(df['Symbol'].unique())
# cutoff_date = str('2014' + "-01-01")
# end_date = str(date.today())

# df_f = quandl.get_table('SHARADAR/SF1', dimension='MRY', calendardate={'gte': cutoff_date, 'lte' : end_date}, ticker=stock_list, paginate=True)
    

In [8]:
#df_f

In [9]:
# Pricing Data Pull from yfinance
def pricing_yfinance_data(fund_dict):
    
    """Pull the pricing data for each ticker.
    Arguments:
    fund_dict -- dict, dictionary where k,v pairs are ticker,fundamental_data pairs
    Returns:
    dict -- keys are tickers
    """

    class HiddenPrints():
        def __enter__(self):
            self._original_stdout = sys.stdout
            sys.stdout = open(os.devnull, 'w')

        def __exit__(self, exc_type, exc_val, exc_tb):
            sys.stdout.close()
            sys.stdout = self._original_stdout

    print("Acquiring pricing data for tickers.")

    total_length = 0

    #current_price = {}
    to_del = []

    # Set business days
    isBusinessDay = BDay().is_on_offset

    # Iterate through dictionary pulling data for that symbol, joining to
    # corresponding fundamentals dataframe.
    for k in fund_dict:
        start_date = min(fund_dict[k]['reportperiod'])
        #end_date = max(fund_dict[k]['reportperiod'])
        end_date = date.today()

        with HiddenPrints():
            yf.pdr_override()
            data = pdr.get_data_yahoo(k, start=start_date, end=end_date)
#            data = yf.download(k, start=start_date, end=end_date)

        if len(data) > 0:
        #    current_price = data['Close'].iloc[-1]
            data.reset_index(inplace=True)
            data.rename(columns={"Date" : "reportperiod"}, inplace=True)

            # Sometimes quarter end/reporting period end is not on trading day.
            # To do this, we will do an outer join of all data, sort by date, backfill data, then eliminate non-trading days
            fund_dict[k] = pd.merge(fund_dict[k], data, left_on='reportperiod', right_on='reportperiod', how='outer', sort=False)
            fund_dict[k].sort_values('reportperiod',ascending=False,inplace=True)
            fund_dict[k] = fund_dict[k].bfill()

            # Filter for just trading days
            match_series = pd.to_datetime(fund_dict[k]['reportperiod']).map(isBusinessDay)
            fund_dict[k] = fund_dict[k][match_series]

            total_length += len(fund_dict[k])

        elif len(data) == 0:
            to_del.append(k)
            pass

    print("Deleting tickers " + str(to_del) + " from dataset due to insufficient pricing data.")
    for k in to_del:
        del fund_dict[k]

    print("total length of fundamentals data dict: " + str(len(fund_dict.keys())))
    print("total length of fundamentals data: " + str(total_length))
    print("S&P Ticker Count after Data Acquisition:  " + str(len(fund_dict.keys())))

    return fund_dict



In [10]:
# Data Collector
def data_collection(timeframe='MRY', ticker=[], start_yr='2012', end_date=None):

    df = sp500_list_retrieval()
    stock_list = list(df['Symbol'].unique())
    print("S&P 500 Ticker Count:  " + str(len(stock_list)))
    
     # Retrieve fundamentals data for all S&P 500 stocks
    stock_dict = fundamentals_data_pull(timeframe=timeframe, ticker=stock_list, start_yr=start_yr, end_date=None)

   # Merge pricing data for all stocks
    stock_dict = pricing_yfinance_data(stock_dict)

    # Final data frame
    final = pd.concat(stock_dict.values(), ignore_index=True)
    final = final.sort_values(by=['reportperiod','ticker'])

    return final

In [11]:
#df = sp500_list_retrieval()

In [12]:
#stock_list = list(df['Symbol'].unique()) 

In [13]:
#fund_dict = fundamentals_data_pull(timeframe='MRY', ticker=stock_list, start_yr='2012', end_date=None)

#fund_dict 

In [14]:
#end_date = str(date.today())
#data = pdr.get_data_yahoo('CAT', start='2014-01-01', end=end_date)



In [15]:
#data 

In [16]:
#data_combined = pricing_yfinance_data(fund_dict)
#data_combined 

In [17]:
# Collect data
df = data_collection(start_yr='2000') 

# get date stamp
dateTimeObj = datetime.now().strftime("%Y-%m-%d_%H.%M.%S")

with open(f'{parentDirectory}/data/raw_data_{dateTimeObj}.pkl', 'wb+') as out:
    pickle.dump(df, out)

Acquiring S&P500 Tickers list.
S&P 500 Ticker Count:  505
Acquiring fundamentals data for tickers.
No data provided for symbol 'AOS'
No data provided for symbol 'ABT'
No data provided for symbol 'ABBV'
No data provided for symbol 'ABMD'
No data provided for symbol 'ACN'
No data provided for symbol 'ATVI'
No data provided for symbol 'ADM'
No data provided for symbol 'ADBE'
No data provided for symbol 'ADP'
No data provided for symbol 'AAP'
No data provided for symbol 'AES'
No data provided for symbol 'AFL'
No data provided for symbol 'A'
No data provided for symbol 'AIG'
No data provided for symbol 'APD'
No data provided for symbol 'AKAM'
No data provided for symbol 'ALK'
No data provided for symbol 'ALB'
No data provided for symbol 'ARE'
No data provided for symbol 'ALGN'
No data provided for symbol 'ALLE'
No data provided for symbol 'LNT'
No data provided for symbol 'ALL'
No data provided for symbol 'GOOGL'
No data provided for symbol 'GOOG'
No data provided for symbol 'MO'
No data pr

No data provided for symbol 'KEY'
No data provided for symbol 'KEYS'
No data provided for symbol 'KMB'
No data provided for symbol 'KIM'
No data provided for symbol 'KMI'
No data provided for symbol 'KLAC'
No data provided for symbol 'KHC'
No data provided for symbol 'KR'
No data provided for symbol 'LHX'
No data provided for symbol 'LH'
No data provided for symbol 'LRCX'
No data provided for symbol 'LW'
No data provided for symbol 'LVS'
No data provided for symbol 'LDOS'
No data provided for symbol 'LEN'
No data provided for symbol 'LLY'
No data provided for symbol 'LNC'
No data provided for symbol 'LIN'
No data provided for symbol 'LYV'
No data provided for symbol 'LKQ'
No data provided for symbol 'LMT'
No data provided for symbol 'L'
No data provided for symbol 'LOW'
No data provided for symbol 'LUMN'
No data provided for symbol 'LYB'
No data provided for symbol 'MTB'
No data provided for symbol 'MRO'
No data provided for symbol 'MPC'
No data provided for symbol 'MKTX'
No data provi

In [18]:
with open(f'{parentDirectory}/data/raw_data_{dateTimeObj}.pkl', 'rb') as f:
    x = pickle.load(f)

In [19]:
df.ticker.unique() 

array(['NKE', nan, 'MSFT', 'PG', 'CSCO', 'AAPL', 'V', 'DIS', 'AXP', 'BA',
       'CAT', 'CVX', 'DD', 'GE', 'GS', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO',
       'MCD', 'MMM', 'MRK', 'PFE', 'TRV', 'TSLA', 'UNH', 'VZ', 'XOM',
       'HD', 'WMT'], dtype=object)

In [20]:
x[x.ticker=='MSFT'].sample(20).sort_values(by='calendardate')

,ticker,dimension,calendardate,datekey,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,...,taxexp,taxliabilities,tbvps,workingcapital,Open,High,Low,Close,Adj Close,Volume
54353,MSFT,MRY,2011-12-31,2011-06-30,2011-07-06,2022-01-25,0.000000e+00,1.087040e+11,9.806925e+10,7.491800e+10,...,4.921000e+09,2.036000e+09,11.234,4.614400e+10,25.969999,26.370001,25.959999,26.330000,20.976379,48744200.0
54172,MSFT,MRY,2011-12-31,2011-06-30,2012-03-23,2022-01-25,0.000000e+00,1.087040e+11,9.806925e+10,7.491800e+10,...,4.921000e+09,2.036000e+09,11.234,4.614400e+10,32.099998,32.110001,31.719999,32.009998,26.025888,35912200.0
54066,MSFT,MRY,2012-12-31,2012-06-30,2012-08-23,2022-01-25,1.422000e+09,1.212710e+11,1.147348e+11,8.508400e+10,...,5.289000e+09,2.682000e+09,12.464,5.239600e+10,30.389999,30.400000,30.080000,30.260000,24.928549,28355600.0
53965,MSFT,MRY,2012-12-31,2012-06-30,2013-01-22,2022-01-25,1.422000e+09,1.212710e+11,1.147348e+11,8.508400e+10,...,5.289000e+09,2.682000e+09,12.464,5.239600e+10,27.299999,27.450001,27.000000,27.150000,22.550278,58650600.0
53855,MSFT,MRY,2012-12-31,2012-06-30,2013-06-28,2022-01-25,1.422000e+09,1.212710e+11,1.147348e+11,8.508400e+10,...,5.289000e+09,2.682000e+09,12.464,5.239600e+10,34.380001,34.790001,34.340000,34.540001,29.128618,65545500.0
53812,MSFT,MRY,2013-12-31,2013-06-30,2013-08-29,2022-01-25,1.743000e+09,1.424310e+11,1.317738e+11,1.014660e+11,...,5.189000e+09,2.301000e+09,14.889,6.404900e+10,32.930000,33.599998,32.799999,33.549999,28.493103,45284700.0
53629,MSFT,MRY,2013-12-31,2013-06-30,2014-05-22,2022-01-25,1.743000e+09,1.424310e+11,1.317738e+11,1.014660e+11,...,5.189000e+09,2.301000e+09,14.889,6.404900e+10,40.290001,40.349998,39.849998,40.099998,34.815315,20201800.0
53358,MSFT,MRY,2014-12-31,2014-06-30,2015-06-19,2022-01-25,3.708000e+09,1.723840e+11,1.560985e+11,1.142460e+11,...,5.746000e+09,3.510000e+09,17.505,6.862100e+10,46.790001,46.830002,45.990002,46.099998,41.082291,63837000.0
53402,MSFT,MRY,2014-12-31,2014-06-30,2015-04-17,2022-01-25,3.708000e+09,1.723840e+11,1.560985e+11,1.142460e+11,...,5.746000e+09,3.510000e+09,17.505,6.862100e+10,41.669998,41.740002,41.160000,41.619999,36.850422,42387600.0
53274,MSFT,MRY,2015-12-31,2015-06-30,2015-10-19,2022-01-25,2.522000e+09,1.744720e+11,1.739148e+11,1.227970e+11,...,6.314000e+09,1.901000e+09,18.674,7.315000e+10,47.419998,47.880001,47.020000,47.619999,42.716698,29387600.0


In [21]:
x.isnull().sum() 

ticker          27
dimension       27
calendardate    27
datekey         27
reportperiod     0
                ..
High             0
Low              0
Close            0
Adj Close        0
Volume           0
Length: 117, dtype: int64

In [22]:
x.columns[0:90] 

Index(['ticker', 'dimension', 'calendardate', 'datekey', 'reportperiod',
       'lastupdated', 'accoci', 'assets', 'assetsavg', 'assetsc', 'assetsnc',
       'assetturnover', 'bvps', 'capex', 'cashneq', 'cashnequsd', 'cor',
       'consolinc', 'currentratio', 'de', 'debt', 'debtc', 'debtnc', 'debtusd',
       'deferredrev', 'depamor', 'deposits', 'divyield', 'dps', 'ebit',
       'ebitda', 'ebitdamargin', 'ebitdausd', 'ebitusd', 'ebt', 'eps',
       'epsdil', 'epsusd', 'equity', 'equityavg', 'equityusd', 'ev', 'evebit',
       'evebitda', 'fcf', 'fcfps', 'fxusd', 'gp', 'grossmargin', 'intangibles',
       'intexp', 'invcap', 'invcapavg', 'inventory', 'investments',
       'investmentsc', 'investmentsnc', 'liabilities', 'liabilitiesc',
       'liabilitiesnc', 'marketcap', 'ncf', 'ncfbus', 'ncfcommon', 'ncfdebt',
       'ncfdiv', 'ncff', 'ncfi', 'ncfinv', 'ncfo', 'ncfx', 'netinc',
       'netinccmn', 'netinccmnusd', 'netincdis', 'netincnci', 'netmargin',
       'opex', 'opinc', 'payables

In [23]:
x.columns[90:] 

Index(['retearn', 'revenue', 'revenueusd', 'rnd', 'roa', 'roe', 'roic', 'ros',
       'sbcomp', 'sgna', 'sharefactor', 'sharesbas', 'shareswa', 'shareswadil',
       'sps', 'tangibles', 'taxassets', 'taxexp', 'taxliabilities', 'tbvps',
       'workingcapital', 'Open', 'High', 'Low', 'Close', 'Adj Close',
       'Volume'],
      dtype='object')

In [24]:
x.columns 

Index(['ticker', 'dimension', 'calendardate', 'datekey', 'reportperiod',
       'lastupdated', 'accoci', 'assets', 'assetsavg', 'assetsc',
       ...
       'taxexp', 'taxliabilities', 'tbvps', 'workingcapital', 'Open', 'High',
       'Low', 'Close', 'Adj Close', 'Volume'],
      dtype='object', length=117)

## Getting data from FMP

In [41]:
def get(url):
    response = urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)

In [45]:
# what financial data we want to get from FMP
#tickers = ["AAPL", "GOOGL", "AMZN", "TSLA"]
tickers = df.ticker.unique() 
ratio_metric = ["returnOnAssets","priceEarningsRatio","inventoryTurnover"]
IS_metric = ["eps","ebitdaratio","revenue"]
BS_metric = ["totalDebt","totalAssets","totalStockholdersEquity"]
CF_metric = ["capitalExpenditure","operatingCashFlow","dividendsPaid"]
Values_metric = ["marketCapitalization"]
Other_metric = ["eps_1yearago","eps_5yearsago","revenue_1yearago","revenue_5yearsago"]
metric = ratio_metric + IS_metric + BS_metric + CF_metric + Values_metric + Other_metric



In [46]:
#metric

In [52]:
df_fmp = pd.DataFrame(index=tickers, columns=metric)

In [49]:
def get_fundamental_data_from_fmp(df):
    for symbol in df.index:
        try:
            url_ratios = ("https://financialmodelingprep.com/api/v3/ratios/" + symbol +"?apikey=eff6c180295d98cddc0606558075bab7")
            url_IS = ("https://financialmodelingprep.com/api/v3/income-statement/" + symbol +"?apikey=eff6c180295d98cddc0606558075bab7")
            url_BS = ("https://financialmodelingprep.com/api/v3/balance-sheet-statement/" + symbol +"?apikey=eff6c180295d98cddc0606558075bab7")
            url_CF = ("https://financialmodelingprep.com/api/v3/cash-flow-statement/" + symbol +"?apikey=eff6c180295d98cddc0606558075bab7")
            url_Values = ("https://financialmodelingprep.com/api/v3/enterprise-values/" + symbol +"?apikey=eff6c180295d98cddc0606558075bab7")
            
            for i in ratio_metric:                
                df.loc[symbol,i] = get(url_ratios)[0][i]   
            for i in IS_metric:
                df.loc[symbol,i] = get(url_IS)[0][i]
            for i in BS_metric:
                df.loc[symbol,i] = get(url_BS)[0][i]
            for i in CF_metric:
                df.loc[symbol,i] = get(url_CF)[0][i]            
            for i in Values_metric:
                df.loc[symbol,i] = get(url_Values)[0][i]     
                
# It is better to write the financial data below in this order (write 1 year ago first, then 5 years ago) because there 
# are some stocks (e.g. Google) which don't have data of EPS 5 years ago. This order shows financial data as much as
# possible because the loop ends once a NaN appears.
            df.loc[symbol,"eps_1yearago"] = get(url_IS)[1]["eps"]
            df.loc[symbol,"revenue_1yearago"] = get(url_IS)[1]["revenue"]
            df.loc[symbol,"eps_5yearsago"] = get(url_IS)[5]["eps"]
            df.loc[symbol,"revenue_5yearsago"] = get(url_IS)[5]["revenue"]
 
        except Exception as e:
            print (symbol, 'not found')
    return df


In [53]:
df_fmp_data = get_fundamental_data_from_fmp(df_fmp)

NKE not found
nan not found
MSFT not found
PG not found
CSCO not found
AAPL not found
V not found
DIS not found
AXP not found
BA not found
CAT not found
CVX not found
DD not found
GE not found
GS not found
IBM not found
INTC not found
JNJ not found
JPM not found
KO not found
MCD not found
MMM not found
MRK not found
PFE not found
TRV not found
TSLA not found
UNH not found
VZ not found
XOM not found
HD not found
WMT not found


In [62]:
df_fmp_data.loc['CSCO']

returnOnAssets                 0.108629
priceEarningsRatio            23.571605
inventoryTurnover             11.497114
eps                                2.51
ebitdaratio                    0.312297
revenue                     49818000000
totalDebt                   11526000000
totalAssets                 97497000000
totalStockholdersEquity     41275000000
capitalExpenditure           -692000000
operatingCashFlow           15454000000
dividendsPaid               -6163000000
marketCapitalization       249646864222
eps_1yearago                   2.647309
eps_5yearsago                       NaN
revenue_1yearago            49301000000
revenue_5yearsago                   NaN
Name: CSCO, dtype: object

In [55]:
x[x.ticker=='CSCO']

,ticker,dimension,calendardate,datekey,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,assetsnc,assetturnover,bvps,capex,cashneq,cashnequsd,cor,consolinc,currentratio,de,debt,debtc,debtnc,debtusd,deferredrev,depamor,deposits,divyield,dps,ebit,ebitda,ebitdamargin,ebitdausd,ebitusd,ebt,eps,epsdil,epsusd,equity,equityavg,equityusd,ev,evebit,evebitda,fcf,fcfps,fxusd,gp,grossmargin,intangibles,intexp,invcap,invcapavg,inventory,investments,investmentsc,investmentsnc,liabilities,liabilitiesc,liabilitiesnc,marketcap,ncf,ncfbus,ncfcommon,ncfdebt,ncfdiv,ncff,ncfi,ncfinv,ncfo,ncfx,netinc,netinccmn,netinccmnusd,netincdis,netincnci,netmargin,opex,opinc,payables,payoutratio,pb,pe,pe1,ppnenet,prefdivis,price,ps,ps1,receivables,retearn,revenue,revenueusd,rnd,roa,roe,roic,ros,sbcomp,sgna,sharefactor,sharesbas,shareswa,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital,Open,High,Low,Close,Adj Close,Volume
18177,CSCO,MRY,2011-12-31,2011-07-30,2011-08-01,2022-02-22,1.294000e+09,8.709500e+10,8.361025e+10,5.723100e+10,2.986400e+10,0.517,8.542,-1.174000e+09,7.662000e+09,7.662000e+09,1.668200e+10,6.490000e+09,3.269,0.844,1.682200e+10,5.880000e+08,1.623400e+10,1.682200e+10,1.220700e+10,2.486000e+09,0.0,0.008,0.12,8.453000e+09,1.093900e+10,0.253,1.093900e+10,8.453000e+09,7.825000e+09,1.17,1.17,1.17,4.722600e+10,4.617625e+10,4.722600e+10,9.795195e+10,12.0,8.954,8.905000e+09,1.611,1.0,2.653600e+10,0.614,1.935900e+10,628000000.0,5.939000e+10,5.671725e+10,1.486000e+09,4.352200e+10,4.003400e+10,3.488000e+09,3.983600e+10,1.750600e+10,2.233000e+10,8.783795e+10,3.081000e+09,-2.660000e+08,-5.065000e+09,1.508000e+09,-6.580000e+08,-4.064000e+09,-2.934000e+09,-1.516000e+09,1.007900e+10,0.0,6.490000e+09,6.490000e+09,6.490000e+09,0.0,0.0,0.150,1.886200e+10,7.674000e+09,8.760000e+08,0.103,1.860,13.534,13.65,3.916000e+09,0.0,15.97,2.032,2.043,4.698000e+09,7.284000e+09,4.321800e+10,4.321800e+10,5.823000e+09,0.078,0.141,0.149,0.196,1.549000e+09,1.172000e+10,1.0,5.500185e+09,5.529000e+09,5.563000e+09,7.817,6.773600e+10,2.410000e+09,1.335000e+09,1.311000e+09,12.251,3.972500e+10,16.080000,16.250000,15.550000,15.830000,11.610582,60652700.0
18176,CSCO,MRY,2011-12-31,2011-07-30,2011-08-02,2022-02-22,1.294000e+09,8.709500e+10,8.361025e+10,5.723100e+10,2.986400e+10,0.517,8.542,-1.174000e+09,7.662000e+09,7.662000e+09,1.668200e+10,6.490000e+09,3.269,0.844,1.682200e+10,5.880000e+08,1.623400e+10,1.682200e+10,1.220700e+10,2.486000e+09,0.0,0.008,0.12,8.453000e+09,1.093900e+10,0.253,1.093900e+10,8.453000e+09,7.825000e+09,1.17,1.17,1.17,4.722600e+10,4.617625e+10,4.722600e+10,9.795195e+10,12.0,8.954,8.905000e+09,1.611,1.0,2.653600e+10,0.614,1.935900e+10,628000000.0,5.939000e+10,5.671725e+10,1.486000e+09,4.352200e+10,4.003400e+10,3.488000e+09,3.983600e+10,1.750600e+10,2.233000e+10,8.783795e+10,3.081000e+09,-2.660000e+08,-5.065000e+09,1.508000e+09,-6.580000e+08,-4.064000e+09,-2.934000e+09,-1.516000e+09,1.007900e+10,0.0,6.490000e+09,6.490000e+09,6.490000e+09,0.0,0.0,0.150,1.886200e+10,7.674000e+09,8.760000e+08,0.103,1.860,13.534,13.65,3.916000e+09,0.0,15.97,2.032,2.043,4.698000e+09,7.284000e+09,4.321800e+10,4.321800e+10,5.823000e+09,0.078,0.141,0.149,0.196,1.549000e+09,1.172000e+10,1.0,5.500185e+09,5.529000e+09,5.563000e+09,7.817,6.773600e+10,2.410000e+09,1.335000e+09,1.311000e+09,12.251,3.972500e+10,15.610000,15.810000,15.450000,15.460000,11.339204,56227200.0
18175,CSCO,MRY,2011-12-31,2011-07-30,2011-08-03,2022-02-22,1.294000e+09,8.709500e+10,8.361025e+10,5.723100e+10,2.986400e+10,0.517,8.542,-1.174000e+09,7.662000e+09,7.662000e+09,1.668200e+10,6.490000e+09,3.269,0.844,1.682200e+10,5.880000e+08,1.623400e+10,1.682200e+10,1.220700e+10,2.486000e+09,0.0,0.008,0.12,8.453000e+09,1.093900e+10,0.253,1.093900e+10,8.453000e+09,7.825000e+09,1.17,1.17,1.17,4.722600e+10,4.617625e+10,4.722600e+10,9.795195e+10,12.0,8.954,8.905000e+09,1.611,1.0,2.653600e+10,0.614,1.935900e+10,628000000.0,5.939000e+10,5.671725e+10,1.486000e+09,4.352200e+10,4.003400e+10,3.488000e+09,3.9